# FVCOM Access Via OpenDap

This notebook shows 3 access points for working with FVCOM data over OpenDap/THREDDS connections to the SMAST Catalog. These access points are for stand-alone 30-year hindcast dataset for Gulf of Maine, individual year and month URL's for archvived output from 2016-2023, and forecast information available 3 days from present.


Following steps outlined here:

Accessing hindcast
https://github.com/iuryt/FVCOMpy/blob/main/examples/00-GettingStarted.ipynb 

Accessing forecast
https://tristansalles.github.io/Coast/queryocean/oceanforecast.html#loading-the-model-dataset

THREDDS Catalog
http://www.smast.umassd.edu:8080/thredds/catalog/models/fvcom/catalog.html 

In [3]:
# Import Libraries
import xarray as xr
import os


#importing modules used as auxiliary
from dateutil import parser
from datetime import datetime,timedelta

# Set directory of where to save it
os.chdir("../local")
os.getcwd()

'/Users/akemberling/Documents/Repositories/casco_bay_mursst/local'

## Approach 1: Forecast

FVCOM Data is available projected forward in time.

In [4]:
# # URL for forecast model
# url = 'http://www.smast.umassd.edu:8080/thredds/dodsC/FVCOM/NECOFS/Forecasts/NECOFS_FVCOM_OCEAN_MASSBAY_FORECAST.nc'

# # list problematic coordinates
# drop_variables = ['siglay','siglev']

# # lazy load of the data
# ds = xr.open_dataset(url,drop_variables=drop_variables,decode_times=False)

# # convert lon/c, lat/c to coordinates
# ds = ds.assign_coords({var:ds[var] for var in ['lon','lat','lonc','latc']})
# ds

## Approach 2: 2016-2023 Hindcast

In [5]:
# # This wasn't working - time period selection

# # define year and month to be read
# year = 2016
# month = 5

# # list problematic coordinates
# drop_variables = ['siglay','siglev']

# # Build URL(s)
# url = "".join([
#         "http://www.smast.umassd.edu:8080/thredds/dodsC/models/fvcom/",
#         f"NECOFS/Archive/NECOFS_GOM/{year}/gom4_{year}{month:02d}.nc?"])

# # lazy load of the data
# ds = xr.open_dataset(url,drop_variables=drop_variables,decode_times=False)

# # convert lon/c, lat/c to coordinates
# ds = ds.assign_coords({var:ds[var] for var in ['lon','lat','lonc','latc']})
# ds

## Approach 3: 30-year Hindcast

This is the catalog option that Matt used for the ecosystem indicators

In [6]:
# 30-year hindcast

# we could use this to run a loop through the years/months we need

# list problematic coordinates
drop_variables = ['siglay','siglev']

# base url for openDAP server for 30-year hindcast
# source: http://www.smast.umassd.edu:8080/thredds/hindcasts.html?dataset=fvcom/hindcasts/30yr_gom3
url = "http://www.smast.umassd.edu:8080/thredds/dodsC/fvcom/hindcasts/30yr_gom3"

# lazy load of the data
ds = xr.open_dataset(url,drop_variables=drop_variables,decode_times=False)

# convert lon/c, lat/c to coordinates
ds = ds.assign_coords({var:ds[var] for var in ['lon','lat','lonc','latc']})
ds

<xarray.Dataset>
Dimensions:        (nele: 90415, node: 48451, three: 3, maxnode: 11,
                    maxelem: 9, four: 4, time: 342348, siglay: 45, siglev: 46)
Coordinates:
    lon            (node) float32 -59.89 -59.81 -59.75 ... -73.95 -73.96 -73.95
    lat            (node) float32 46.09 46.15 46.09 46.04 ... 41.62 41.62 41.62
    lonc           (nele) float32 -59.86 -59.84 -59.8 ... -73.96 -73.95 -73.95
    latc           (nele) float32 46.05 46.1 46.1 46.06 ... 41.62 41.61 41.62
  * time           (time) float32 4.351e+04 4.351e+04 ... 5.775e+04 5.775e+04
Dimensions without coordinates: nele, node, three, maxnode, maxelem, four,
                                siglay, siglev
Data variables: (12/45)
    nprocs         int32 ...
    partition      (nele) int32 ...
    x              (node) float32 ...
    y              (node) float32 ...
    xc             (nele) float32 ...
    yc             (nele) float32 ...
    ...             ...
    l              (time, siglev, node) float32 ...
    short_wave     (time, node) float32 ...
    net_heat_flux  (time, node) float32 ...
    uwind_stress   (time, nele) float32 ...
    vwind_stress   (time, nele) float32 ...
    fvcom_mesh     int32 ...
Attributes: (12/17)
    title:                       GOM3 Nesting  by Yf.Sun@umassd.edu
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_3.0
    history:                     model started at: 22/12/2011   16:41
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.4, UGRID-0.9
    ...                          ...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...
    cdm_data_type:               any
    DODS.strlen:                 26
    DODS.dimName:                DateStrLen

###  Fixing Dates

Use the 30-year hindcast as a test of the date-repair routine. This should tell us what range of dates we have and whether this dataset willl work or not:

In [7]:
# Fix time to see what the thirty years is:

# the first day
dt0 = parser.parse(ds.time.attrs['units'].replace('days since ',''))

# parse dates summing days to the origin
ds = ds.assign(time = [dt0 + timedelta(seconds = day * 86400) for day in ds.time.values])
ds.time

<xarray.DataArray 'time' (time: 342348)>
array(['1978-01-01T00:00:00.000000000', '1978-01-01T01:01:52.500000000',
       '1978-01-01T01:58:07.500000000', ..., '2016-12-31T22:01:52.500000000',
       '2016-12-31T22:58:07.500000000', '2017-01-01T00:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1978-01-01 ... 2017-01-01

In [13]:
# Can we plot anything
ds.isel(time = 0).

<xarray.Dataset>
Dimensions:        (nele: 90415, node: 48451, three: 3, maxnode: 11,
                    maxelem: 9, four: 4, siglay: 45, siglev: 46)
Coordinates:
    lon            (node) float32 -59.89 -59.81 -59.75 ... -73.95 -73.96 -73.95
    lat            (node) float32 46.09 46.15 46.09 46.04 ... 41.62 41.62 41.62
    lonc           (nele) float32 -59.86 -59.84 -59.8 ... -73.96 -73.95 -73.95
    latc           (nele) float32 46.05 46.1 46.1 46.06 ... 41.62 41.61 41.62
    time           datetime64[ns] 1978-01-01
Dimensions without coordinates: nele, node, three, maxnode, maxelem, four,
                                siglay, siglev
Data variables: (12/45)
    nprocs         int32 48
    partition      (nele) int32 35 35 35 35 35 35 35 35 ... 10 10 10 10 10 10 10
    x              (node) float32 1.694e+06 1.7e+06 ... 5.841e+05 5.848e+05
    y              (node) float32 4.14e+05 4.205e+05 ... -1.279e+05 -1.28e+05
    xc             (nele) float32 1.697e+06 1.698e+06 ... 5.848e+05 5.845e+05
    yc             (nele) float32 4.097e+05 4.15e+05 ... -1.284e+05 -1.281e+05
    ...             ...
    l              (siglev, node) float32 ...
    short_wave     (node) float32 ...
    net_heat_flux  (node) float32 ...
    uwind_stress   (nele) float32 ...
    vwind_stress   (nele) float32 ...
    fvcom_mesh     int32 -2147483647
Attributes: (12/17)
    title:                       GOM3 Nesting  by Yf.Sun@umassd.edu
    institution:                 School for Marine Science and Technology
    source:                      FVCOM_3.0
    history:                     model started at: 22/12/2011   16:41
    references:                  http://fvcom.smast.umassd.edu, http://codfis...
    Conventions:                 CF-1.4, UGRID-0.9
    ...                          ...
    Surface_Heat_Forcing:        FVCOM variable surface heat forcing file:\nF...
    Surface_Wind_Forcing:        FVCOM variable surface Wind forcing:\nFILE N...
    Surface_PrecipEvap_Forcing:  FVCOM periodic surface precip forcing:\nFILE...
    cdm_data_type:               any
    DODS.strlen:                 26
    DODS.dimName:                DateStrLen

## Explore 30-year

In [ ]:
# Variables of interest are u, ua, v, & va
